In [6]:
import os 
from pathlib import Path 
import pandas as pd 
result_dir = 'results/'
t5_fname = 't5_diaasq_speaker_en_inference_20230525-141254.csv'
gpt_fname = 'SpeakerDiaAsqDataset_gpt_eval.csv'



In [7]:

t5_res = pd.read_csv(os.path.join(result_dir, t5_fname))
gpt_res = pd.read_csv(os.path.join(result_dir, gpt_fname))

In [8]:
t5_res.head()

,doc_id,speaker,pred,label
0,2,A,"(XPhone,experienceor,noter,n) (Xiaomi 11,exper...","(iPhone,processor,better,pos) (Xiaomi 11,photo..."
1,2,B,"(iPhone,Sor,excellent,pos) (iPhone,iOS,beaten,...","(iPhone,processor,excellent,pos) (iPhone,iOS,e..."
2,2,C,"(iPhone,Sales,beat,pos) (","(iPhone,Sales,beat,pos)"
3,2,D,"(Xi 11,photos, blurlurry,neg) (Xi 11,photo,spe...","(Mi 11,photos,blurry,neg) (Mi 11,experience,sp..."
4,12,A,"(10U,telephoto,not outdated,pos)","(10U,telephoto,not outdated,pos)"


In [9]:
gpt_res.head()

,input_sequence,doc_id,speaker,label,pred
0,Sentiment Analysis\nA sentiment tuple consists...,2,A,"(iPhone,processor,better,pos)\n(Xiaomi 11,phot...","(iPhone,processor,except for,neg)\n(Xiaomi,nul..."
1,Sentiment Analysis\nA sentiment tuple consists...,2,B,"(iPhone,processor,excellent,pos)\n(iPhone,iOS,...","(iPhone,processor,excellent,pos)\n(iPhone,iOS,..."
2,Sentiment Analysis\nA sentiment tuple consists...,2,C,"(iPhone,Sales,beat,pos)\n","(iPhone,processor,excellent,pos)\n(iPhone,iOS,..."
3,Sentiment Analysis\nA sentiment tuple consists...,2,D,"(Mi 11,photos,blurry,neg)\n(Mi 11,experience,s...","(null,X,n't used,neg)\n(X,null,good photos,neg..."
4,Sentiment Analysis\nA sentiment tuple consists...,12,A,"(10U,telephoto,not outdated,pos)\n","(null,10U,not outdated,pos)"


In [20]:
# turn null into empty string
t5_res['pred'] = t5_res['pred'].fillna('')
gpt_res['pred'] = gpt_res['pred'].fillna('')
gpt_res['label'] = gpt_res['label'].fillna('')
t5_res['label'] = t5_res['label'].fillna('')

# create doc_id and speaker as 1 col 
t5_res['doc_id'] = t5_res['doc_id'].astype(str)
t5_res['speaker'] = t5_res['speaker'].astype(str)
t5_res['doc_id_speaker'] = t5_res['doc_id'] + '_' + t5_res['speaker']

gpt_res['doc_id'] = gpt_res['doc_id'].astype(str)
gpt_res['speaker'] = gpt_res['speaker'].astype(str)
gpt_res['doc_id_speaker'] = gpt_res['doc_id'] + '_' + gpt_res['speaker']

# merge by doc_id_speaker
merged_res = pd.merge(t5_res, gpt_res, on='doc_id_speaker', how='outer', suffixes=('_t5', '_gpt'))
merged_res.head()


,doc_id_t5,speaker_t5,pred_t5,label_t5,doc_id_speaker,input_sequence,doc_id_gpt,speaker_gpt,label_gpt,pred_gpt
0,2,A,"(XPhone,experienceor,noter,n) (Xiaomi 11,exper...","(iPhone,processor,better,pos) (Xiaomi 11,photo...",2_A,Sentiment Analysis\nA sentiment tuple consists...,2,A,"(iPhone,processor,better,pos)\n(Xiaomi 11,phot...","(iPhone,processor,except for,neg)\n(Xiaomi,nul..."
1,2,B,"(iPhone,Sor,excellent,pos) (iPhone,iOS,beaten,...","(iPhone,processor,excellent,pos) (iPhone,iOS,e...",2_B,Sentiment Analysis\nA sentiment tuple consists...,2,B,"(iPhone,processor,excellent,pos)\n(iPhone,iOS,...","(iPhone,processor,excellent,pos)\n(iPhone,iOS,..."
2,2,C,"(iPhone,Sales,beat,pos) (","(iPhone,Sales,beat,pos)",2_C,Sentiment Analysis\nA sentiment tuple consists...,2,C,"(iPhone,Sales,beat,pos)\n","(iPhone,processor,excellent,pos)\n(iPhone,iOS,..."
3,2,D,"(Xi 11,photos, blurlurry,neg) (Xi 11,photo,spe...","(Mi 11,photos,blurry,neg) (Mi 11,experience,sp...",2_D,Sentiment Analysis\nA sentiment tuple consists...,2,D,"(Mi 11,photos,blurry,neg)\n(Mi 11,experience,s...","(null,X,n't used,neg)\n(X,null,good photos,neg..."
4,12,A,"(10U,telephoto,not outdated,pos)","(10U,telephoto,not outdated,pos)",12_A,Sentiment Analysis\nA sentiment tuple consists...,12,A,"(10U,telephoto,not outdated,pos)\n","(null,10U,not outdated,pos)"


In [24]:
from metrics.utils import str_2_tuples
from typing import List, Tuple




pair_res = {} 
for idx, row in merged_res.iterrows():
    
    t5_res_quads = str_2_tuples(row['pred_t5'])
    gpt_res_quads = str_2_tuples(row['pred_gpt'])
    label_quads = str_2_tuples(row['label_t5']) # or label_gpt, they contain same quads
    id = row['doc_id_speaker']
    pair_res[id] = {
        't5': t5_res_quads,
        'gpt': gpt_res_quads,
        'label': label_quads, 
        'input': row['input_sequence']
    }


    

In [26]:
pair_res['12_A']
# The 10U telephoto is still not outdated now . [ DOGE ] .

{'t5': [('10U', 'telephoto', 'not outdated', 'pos')],
 'gpt': [('null', '10U', 'not outdated', 'pos')],
 'label': [('10U', 'telephoto', 'not outdated', 'pos')],
 'input': 'Sentiment Analysis\nA sentiment tuple consists of (target term, aspect term, opinion term, sentiment polarity).\nGiven a discussion thread associated with a speaker, extract the sentiment tuples that belong to the specified speaker, which means that the "opinion term" of the sentiment tuple is spoken by the speaker.\nThe first 3 elements in the tuple are term-extraction tasks, meaning the extracted string must be the substring of one of the sentences.\nSentiment polarity shall be selected from [pos, neg, other].\nThe output of the tuple is wrapped in (), separated by a half-width comma, and the output of the tuple is separated by \'\\n\'./nInput: \nThe X from 3 years ago , aftre update to the latest 14.2 , it was smoother than the Mi 10 in my hand . It was no problem for Android to ues 3 years?You need to ensure that

In [29]:
id = '22_B'

print(pair_res[id]['input'])
pair_res[id]

Sentiment Analysis
A sentiment tuple consists of (target term, aspect term, opinion term, sentiment polarity).
Given a discussion thread associated with a speaker, extract the sentiment tuples that belong to the specified speaker, which means that the "opinion term" of the sentiment tuple is spoken by the speaker.
The first 3 elements in the tuple are term-extraction tasks, meaning the extracted string must be the substring of one of the sentences.
Sentiment polarity shall be selected from [pos, neg, other].
The output of the tuple is wrapped in (), separated by a half-width comma, and the output of the tuple is separated by '\n'./nInput: 
The X from 3 years ago , aftre update to the latest 14.2 , it was smoother than the Mi 10 in my hand . It was no problem for Android to ues 3 years?You need to ensure that the Android system can be updated for 3 years .
Output: 
(X,null,smoother,pos)
(Mi 10,null,smoother,neg)
(Android,null,no problem,other)
/nInput: 
Sorry , even if the Samsung diamo

{'t5': [('Samsung diamondniameter', 'morelicate', 'pos', '')],
 'gpt': [('Samsung diamond', 'null', 'more delicate', 'pos'),
  ('Zhou Dongyu', 'null', 'more delicate', 'neg')],
 'label': [('Samsung', 'diamond', 'delicate', 'pos')],
 'input': 'Sentiment Analysis\nA sentiment tuple consists of (target term, aspect term, opinion term, sentiment polarity).\nGiven a discussion thread associated with a speaker, extract the sentiment tuples that belong to the specified speaker, which means that the "opinion term" of the sentiment tuple is spoken by the speaker.\nThe first 3 elements in the tuple are term-extraction tasks, meaning the extracted string must be the substring of one of the sentences.\nSentiment polarity shall be selected from [pos, neg, other].\nThe output of the tuple is wrapped in (), separated by a half-width comma, and the output of the tuple is separated by \'\\n\'./nInput: \nThe X from 3 years ago , aftre update to the latest 14.2 , it was smoother than the Mi 10 in my hand

In [30]:
id = '22_D'

print(pair_res[id]['input'])
pair_res[id]
# t5:  ('Andle', 'bull', 'has brother', 'pos')], Andle 是啥看不懂 = = 
# bull 是 null 寫錯
# -> t5 嚴謹地 extraction 能力很差

Sentiment Analysis
A sentiment tuple consists of (target term, aspect term, opinion term, sentiment polarity).
Given a discussion thread associated with a speaker, extract the sentiment tuples that belong to the specified speaker, which means that the "opinion term" of the sentiment tuple is spoken by the speaker.
The first 3 elements in the tuple are term-extraction tasks, meaning the extracted string must be the substring of one of the sentences.
Sentiment polarity shall be selected from [pos, neg, other].
The output of the tuple is wrapped in (), separated by a half-width comma, and the output of the tuple is separated by '\n'./nInput: 
The X from 3 years ago , aftre update to the latest 14.2 , it was smoother than the Mi 10 in my hand . It was no problem for Android to ues 3 years?You need to ensure that the Android system can be updated for 3 years .
Output: 
(X,null,smoother,pos)
(Mi 10,null,smoother,neg)
(Android,null,no problem,other)
/nInput: 
If you want to change Android thi

{'t5': [('Approid', 'F', 'shrunk', 'neg'),
  ('Apple', 's', 'battery life', 'best'),
  ('Android', 'b', 'worse', ' the last'),
  ('Andle', 'bull', 'has brother', 'pos')],
 'gpt': [('Android', 'screen', 'shrunk', 'neg'),
  ('Android', 'processor', 'shrunk', 'neg'),
  ('Android', 'system', 'shrunk', 'neg'),
  ('iOS', 'system', 'best', 'pos'),
  ('Apple', 'battery life', 'best', 'pos'),
  ('Android', 'battery life', 'worse', 'neg'),
  ('Apple', 'null', 'no affect', 'other'),
  ('Android', 'null', 'no affect', 'other'),
  ('Apple', 'null', 'big brother', 'pos'),
  ('Android', 'null', 'delivery boy disappeared', 'neg')],
 'label': [('Android', 'image', 'shrunk', 'neg'),
  ("Apple's", 'battery life', 'best in the world', 'pos'),
  ('Android', 'life', 'worse than the last', 'neg'),
  ('Apple', 'null', 'big brother', 'pos')],
 'input': 'Sentiment Analysis\nA sentiment tuple consists of (target term, aspect term, opinion term, sentiment polarity).\nGiven a discussion thread associated with a sp

In [31]:
id = '42_A'

print(pair_res[id]['input'])
pair_res[id]
# t5, gpt 都知道 no input no output 

Sentiment Analysis
A sentiment tuple consists of (target term, aspect term, opinion term, sentiment polarity).
Given a discussion thread associated with a speaker, extract the sentiment tuples that belong to the specified speaker, which means that the "opinion term" of the sentiment tuple is spoken by the speaker.
The first 3 elements in the tuple are term-extraction tasks, meaning the extracted string must be the substring of one of the sentences.
Sentiment polarity shall be selected from [pos, neg, other].
The output of the tuple is wrapped in (), separated by a half-width comma, and the output of the tuple is separated by '\n'./nInput: 
The X from 3 years ago , aftre update to the latest 14.2 , it was smoother than the Mi 10 in my hand . It was no problem for Android to ues 3 years?You need to ensure that the Android system can be updated for 3 years .
Output: 
(X,null,smoother,pos)
(Mi 10,null,smoother,neg)
(Android,null,no problem,other)
/nInput: 

Output: 



{'t5': [],
 'gpt': [],
 'label': [],
 'input': 'Sentiment Analysis\nA sentiment tuple consists of (target term, aspect term, opinion term, sentiment polarity).\nGiven a discussion thread associated with a speaker, extract the sentiment tuples that belong to the specified speaker, which means that the "opinion term" of the sentiment tuple is spoken by the speaker.\nThe first 3 elements in the tuple are term-extraction tasks, meaning the extracted string must be the substring of one of the sentences.\nSentiment polarity shall be selected from [pos, neg, other].\nThe output of the tuple is wrapped in (), separated by a half-width comma, and the output of the tuple is separated by \'\\n\'./nInput: \nThe X from 3 years ago , aftre update to the latest 14.2 , it was smoother than the Mi 10 in my hand . It was no problem for Android to ues 3 years?You need to ensure that the Android system can be updated for 3 years .\nOutput: \n(X,null,smoother,pos)\n(Mi 10,null,smoother,neg)\n(Android,null,

In [32]:
id = '62_B'

print(pair_res[id]['input'])
pair_res[id]
# gold: ('Xiaomi 11u', 'camera', 'not as good-looking', 'neg'),
# gpt: ('11', 'null', 'good-looking', 'pos'), completely reverse the polarity since it ignores not as negation
# t5: ('11', 'nera', 'not as good-looking', 'pos'), opn term is correct but polarity is wrong, why... 

Sentiment Analysis
A sentiment tuple consists of (target term, aspect term, opinion term, sentiment polarity).
Given a discussion thread associated with a speaker, extract the sentiment tuples that belong to the specified speaker, which means that the "opinion term" of the sentiment tuple is spoken by the speaker.
The first 3 elements in the tuple are term-extraction tasks, meaning the extracted string must be the substring of one of the sentences.
Sentiment polarity shall be selected from [pos, neg, other].
The output of the tuple is wrapped in (), separated by a half-width comma, and the output of the tuple is separated by '\n'./nInput: 
The X from 3 years ago , aftre update to the latest 14.2 , it was smoother than the Mi 10 in my hand . It was no problem for Android to ues 3 years?You need to ensure that the Android system can be updated for 3 years .
Output: 
(X,null,smoother,pos)
(Mi 10,null,smoother,neg)
(Android,null,no problem,other)
/nInput: 
Is the big camera of Xiaomi 11u g

{'t5': [('Xiaomi 11u', 'camera', 'good-looking', 'other'),
  ('11iaomi 11u', 'camera', 'noto big', 'neg'),
  ('11iaomi 11u', 'whiteera', 'not as good-looking asneg', ''),
  ('11', 'nera', 'not as good-looking', 'pos')],
 'gpt': [('X', 'camera', 'good-looking', 'neg'),
  ('11u', 'camera', 'big', 'neg'),
  ('11', 'null', 'good-looking', 'pos'),
  ('null', 'white ceramics', 'like', 'pos')],
 'label': [('Xiaomi 11u', 'camera', 'good-looking', 'other'),
  ('Xiaomi 11u', 'camera', 'too big', 'neg'),
  ('Xiaomi 11u', 'camera', 'not as good-looking', 'neg'),
  ('11', 'camera', 'not as good-looking', 'pos')],
 'input': 'Sentiment Analysis\nA sentiment tuple consists of (target term, aspect term, opinion term, sentiment polarity).\nGiven a discussion thread associated with a speaker, extract the sentiment tuples that belong to the specified speaker, which means that the "opinion term" of the sentiment tuple is spoken by the speaker.\nThe first 3 elements in the tuple are term-extraction tasks, m